In [99]:
from selenium import webdriver
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
import en_core_web_lg
import datetime
from dateutil.parser import parse

### PART A - Extracing Book Reviews

In [128]:
### Scraping book reviewa via Selenium ###

driver = webdriver.Chrome(executable_path='/Users/livestrong19/Downloads/chromedriver')
driver.get('https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once')

### Fetch names of 100 books and URL link

book_dict={}
for i in range(1,101):
    book_path=driver.find_element_by_xpath('//div[@id="all_votes"]/table/tbody/tr['+str(i)+']/td[3]/a')
    book_name=driver.find_element_by_xpath('//div[@id="all_votes"]/table/tbody/tr['+str(i)+']/td[3]/a/span').text
    book_dict[book_name]=book_path.get_attribute('href')
driver.close()

In [129]:
book_dict

{'To Kill a Mockingbird (To Kill a Mockingbird, #1)': 'https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird',
 "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)": 'https://www.goodreads.com/book/show/3.Harry_Potter_and_the_Sorcerer_s_Stone',
 '1984': 'https://www.goodreads.com/book/show/5470.1984',
 'Pride and Prejudice': 'https://www.goodreads.com/book/show/1885.Pride_and_Prejudice',
 'The Diary of a Young Girl': 'https://www.goodreads.com/book/show/48855.The_Diary_of_a_Young_Girl',
 'The Little Prince': 'https://www.goodreads.com/book/show/157993.The_Little_Prince',
 'The Great Gatsby': 'https://www.goodreads.com/book/show/4671.The_Great_Gatsby',
 'The Catcher in the Rye': 'https://www.goodreads.com/book/show/5107.The_Catcher_in_the_Rye',
 'Animal Farm': 'https://www.goodreads.com/book/show/14060211-animal-farm',
 'The Lord of the Rings (The Lord of the Rings, #1-3)': 'https://www.goodreads.com/book/show/33.The_Lord_of_the_Rings',
 'Jane Eyre': 'https://www.goodread

In [ ]:
### Fetch first 30 reviews for each of the 100 books

book_ratings = pd.DataFrame(columns = ['Book Name','User Rating','Book Review', 'Book Shelf']) 

for key,value in book_dict.items():
    driver = webdriver.Chrome('/Users/livestrong19/Downloads/chromedriver')
    driver.get(value)
    for i in range(1,31):
        try:
            reviews = driver.find_element_by_xpath('//div[@id="bookReviews"]/div['+str(i)+']/div/div/div/div[2]/span/span[2]')
            review_text = reviews.get_attribute('innerHTML')
            rating = driver.find_element_by_xpath('//div[@id="bookReviews"]/div['+str(i)+']/div/div/div/div/span[2]')
            user_rating = rating.get_attribute('title')
            shelf = driver.find_element_by_xpath('//div[@id="bookReviews"]/div['+str(i)+']/div/div/div/div/span[2]')
            shelf = driver.find_elements_by_xpath('//div[@id="bookReviews"]/div['+str(i)+']/div/div/div/div[1]/div/a')
            book_shelf = ""
            for x in range(len(shelf)):
                book_shelf = book_shelf + " " + shelf[x].text
            book_ratings.loc[len(book_ratings)] = [key,user_rating,review_text,book_shelf]
        except:
            pass
    driver.close()

In [127]:
book_ratings

,Book Name,User Rating,Book Review,Book Shelf
0,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",it was amazing,Why is it when I pick up <i> To Kill A Mocking...,cultured simpatico mmix
1,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",it was amazing,"<a href=""http://s153.photobucket.com/albums/s2...",all-time-favorites classics-americas easton-p...
2,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",it was amazing,With endless books and infinitely more to be w...,classics rereads southern-lit pulitzer-winner...
3,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",it was amazing,<b>If you haven't read this as an adult - <u>p...,audiobook
4,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",it was amazing,"To Kill a Mockingbird, Harper Lee<br>To Kill a...",academic novel fiction classic historical 20t...
5,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",it was amazing,"In the course of 5 years, I’ve read this book ...",american novels literary fiction favorites owned
6,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",really liked it,So... I don't really know what to say.<br><br>...,2015
7,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",it was ok,<b>2.5 stars</b><br><br>Bestseller. Pulitzer P...,disappointing library historical-fiction scho...
8,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",it was amazing,"“Until I feared I would lose it, I never loved...",
9,"To Kill a Mockingbird (To Kill a Mockingbird, #1)",it was amazing,"English (<a href=""https://www.goodreads.com/bo...",


In [6]:
df = pd.read_csv("booksdata.csv")

In [7]:
df.shape

(31246, 18)

In [8]:
df.columns

Index(['web-scraper-order', 'web-scraper-start-url', 'bookreview_user',
       'booktitle', 'bookauthor', 'booksummary', 'booklength', 'books',
       'books-href', 'bookgenre', 'bookavgrating', 'bookratingcount',
       'bookcover-src', 'publishdate', 'bookreviewcount', 'bookreview_rating',
       'bookreview_1', 'bookreview_2'],
      dtype='object')

Here `bookreview_2` is just the complete version of `bookreview_1` after clicking on more to see hidden text. For some reviews it may be nan because the review was complete in `bookreview_1` only.

### Cleaning Features

In [124]:
def cleanColumns(train):
    today = datetime.date.today()
    train=df.drop(['web-scraper-order','web-scraper-start-url','books-href','bookreview_user'], axis=1)
    genre=train.bookgenre.apply(lambda x: x.replace('[{"bookgenre":','')) \
                    .apply(lambda x: x.replace('{"bookgenre":','')) \
                    .apply(lambda x: x.replace('}','')) \
                    .apply(lambda x: x.replace(']','')) \
                    .apply(lambda x: x.replace('"',''))

    ratecount=train.bookratingcount.apply(lambda x: int(x.split('\n')[0].strip().replace(',','')))
    reviewcount=train.bookreviewcount.apply(lambda x: int(x.split('\n')[0].strip().replace(',','')))
    published=train.publishdate.apply(lambda x: x.split('\n')[1].replace('th','').replace('rd','').replace('nd','').strip())

    
    train.bookgenre=genre
    train.bookratingcount=ratecount
    train.bookreviewcount=reviewcount
    train.publishdate=published
    delta=train.publishdate.apply(lambda x: int( (today - parse(x).date()).days ))
    train.publishdate=delta
    
    
    return train

In [125]:
train=df.copy()
train=cleanColumns(train)

In [126]:
train.head()

,booktitle,bookauthor,booksummary,booklength,books,bookgenre,bookavgrating,bookratingcount,bookcover-src,publishdate,bookreviewcount,bookreview_rating,bookreview_1,bookreview_2
0,The President Is Missing,Bill Clinton,President Bill Clinton and bestselling novelis...,513 pages,31,"Fiction,Thriller,Mystery,Politics,Audiobook",3.83,73745,https://i.gr-assets.com/images/S/compressed.ph...,544,8749,NaN,"So, you may already know that I like to point ...","So, you may already know that I like to point ..."
1,The Institute,Stephen King,"In the middle of the night, in a house on a qu...",561 pages,5,"Horror,Fiction,Thriller,Fantasy,Science Fiction",4.23,42323,https://i.gr-assets.com/images/S/compressed.ph...,81,5575,it was amazing,This is the first Stephen King book I’ve ever ...,This is the first Stephen King book I’ve ever ...
2,"A Higher Loyalty: Truth, Lies, and Leadership",James Comey,Former FBI Director James Comey shares his nev...,290 pages,71,"Nonfiction,Politics,Autobiography,Memoir,Biogr...",4.18,34983,https://i.gr-assets.com/images/S/compressed.ph...,592,4850,it was ok,Meh... I'm a bit of a news junkie and there wa...,NaN
3,"Option B: Facing Adversity, Building Resilienc...",Sheryl Sandberg,From Facebook's COO and Wharton's top-rated pr...,240 pages,92,"Nonfiction,Self Help,Psychology,Autobiography,...",3.87,39146,https://i.gr-assets.com/images/S/compressed.ph...,950,3261,it was amazing,I have always felt like I was pretty resilient...,I have always felt like I was pretty resilient...
4,The Next Person You Meet in Heaven,Mitch Albom,In this enchanting sequel to the number one be...,224 pages,42,"Fiction,Inspirational,Contemporary",4.29,19270,https://i.gr-assets.com/images/S/compressed.ph...,417,2458,it was amazing,Mitch Albom's sequel is as engaging. Heaven wa...,Mitch Albom's sequel is as engaging. Heaven wa...


## Google Vision Api

In [ ]:
from google.cloud import vision
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "keyFile.json"
image_urls = list_of_urls
ls_of_label_for_url = []

for url in image_urls:
    image_uri = url
    label_list = []
    client = vision.ImageAnnotatorClient()
    image = vision.types.Image()
    image.source.image_uri = image_uri
    response = client.label_detection(image=image)
    print('Labels (and confidence score):')
    print('=' * 79)
    
    for label in response.label_annotations:
        label_list.append(label.description)
        print(f'{label.description} ({label.score*100.:.2f}%)')
    ls_of_label_for_url.append(label_list)
    
print(ls_of_label_for_url)
print('Length of unedited list',len(ls_of_label_for_url))

ls_of_label_for_url = list(filter(None, ls_of_label_for_url))
print(ls_of_label_for_url)
print('Length of edited list', len(ls_of_label_for_url))